# Accessing FDA Recalls

In [20]:
# Import packages
import requests
import pandas as pd
import numpy as np

In [62]:
def fetch_data(base_url, params):
    all_data = []
    while True:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            if 'results' in data:
                all_data.extend(data['results'])
                params['skip'] += params['limit'] 
            else:
                print("No more results found.")
                break
        elif response.status_code == 404:
            if len(all_data)>0:
                print(f"{len(all_data)} results found - No more results available.")
                break
            else:
                print("No matches found. Check your query parameters.")
                break
        else:
            print("Failed to retrieve data:", response.status_code, response.text)
            break
    return all_data

# Instantiate search variables and parameters
base_url = 'https://api.fda.gov/food/enforcement.json'
start_date = '20240101'
end_date = '20240531'
query_params = {
    'search': f'recall_initiation_date:["{start_date}" TO "{end_date}"]',
    'limit': 100,
    'skip': 0 
}

# Fetch all data
data = fetch_data(base_url, query_params)

# Create DataFrame from the collected data
df = pd.json_normalize(data)

# Convert date columns
date_columns = ['recall_initiation_date', 'center_classification_date', 'termination_date', 'report_date']
for column in date_columns:
    df[column] = pd.to_datetime(df[column], format='%Y%m%d', errors='coerce')

378 results found - No more results available.


In [68]:
# Inspect results
print('Min date:', df.recall_initiation_date.min())
print('Max date:', df.recall_initiation_date.max())
print(df.columns)
df.head(2)

Min date: 2024-01-02 00:00:00
Max date: 2024-05-17 00:00:00
Index(['status', 'city', 'state', 'country', 'classification', 'product_type',
       'event_id', 'recalling_firm', 'address_1', 'address_2', 'postal_code',
       'voluntary_mandated', 'initial_firm_notification',
       'distribution_pattern', 'recall_number', 'product_description',
       'product_quantity', 'reason_for_recall', 'recall_initiation_date',
       'center_classification_date', 'report_date', 'code_info',
       'more_code_info', 'termination_date'],
      dtype='object')


,status,city,state,country,classification,product_type,event_id,recalling_firm,address_1,address_2,...,recall_number,product_description,product_quantity,reason_for_recall,recall_initiation_date,center_classification_date,report_date,code_info,more_code_info,termination_date
0,Ongoing,San Luis,AZ,United States,Class I,Food,93920,HandNatural,874 S Main St,,...,F-1170-2024,"H&NATURAL 2 PACK! BRAZIL SEED 60 PIECES, PURE ...",,Product recalled due to the presence of yellow...,2024-02-09,2024-04-16,2024-04-24,No Lot code on label. Expiration date listed a...,,NaT
1,Ongoing,Cleveland,OH,United States,Class II,Food,93817,Greco Sidari,3820 Lakeside Ave E,,...,F-0848-2024,GRANDE 1.5OZ FRESH BOCCONCINI MOZZ 1/3# ...,1375 cases total,The food was not held at an appropriate temper...,2024-01-11,2024-02-05,2024-02-14,"All Lot Codes Distributed between November 26,...",,NaT


In [69]:
df.status.value_counts()

status
Ongoing       226
Terminated    134
Completed      18
Name: count, dtype: int64

In [70]:
df.product_description.to_list()[:10]

['H&NATURAL 2 PACK! BRAZIL SEED 60 PIECES, PURE NATURAL SEMILLA DE BRASIL FOR 60 DAYS, 5 GRAMS PER BOX, 2 BLACK BOXES.',
 'GRANDE     1.5OZ FRESH BOCCONCINI MOZZ 1/3#         SUPC/ITEM # 19132P',
 'Sysco Classic Riced Cauliflower packed in a 4lb plastic bag (6 units of 4 lb. per carton box).',
 'Vanilla Ice Cream Cookie Sandwich. Sandwiches packaged in a gold or clear bags. 50 sandwiches per case and weighting 4oz each sandwich.',
 'California Wine Wafer - Mocha Chocolate and Original Wine Wafer - Mocha Chocolate [Two sizes: retail package of 8 wafers (7 oz) and retail gift package of 2 wafers (2 oz)]',
 'Fresh Local , Country Corner Dairy,  YELLOW COLBY CHEESE, INGREDIENTS: Pasteurized Milk, Salt , Rennet, Calcium Chloride,  Cheese Culture, Contains Milk. Product comes in various sizes, 6 oz. up to 42 lbs. and is in vacuum sealed plastic packaging.',
 'SAPUTO STELLA ASGO HLF MLD 11LB  SUPC/ITEM # 19009',
 'Dannon Plain Yogurt packaged in 32oz plastic carton with lid - 6 cartons per ca